In [1]:
import argparse
import os
import shutil
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain_chroma import Chroma

CHROMA_PATH = "chroma"
DATA_PATH = "data" 

# Document LOADER
def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

# Split Documents into Chunks
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

# DATABANK
def add_to_chroma(chunks: list[Document]):
    # Load the existing database.
    db = Chroma(
        persist_directory=CHROMA_PATH, embedding_function=get_embedding_function()
    )

    # Calculate Page IDs.
    chunks_with_ids = calculate_chunk_ids(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        
    else:
        print("✅ No new documents to add")


def calculate_chunk_ids(chunks):

    # This will create IDs like "data/SILOP.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks


def clear_database():
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

from langchain_ollama import OllamaEmbeddings


def get_embedding_function():
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    return embeddings

In [2]:
# Create (or update) the data store.

#print("✨ Clearing Database")
#clear_database()


documents = load_documents()
chunks = split_documents(documents)
add_to_chroma(chunks)

Number of existing documents in DB: 0
👉 Adding new documents: 428


In [3]:
from langchain_ollama import OllamaLLM
from langchain.prompts import ChatPromptTemplate
CHROMA_PATH = "chroma"


def query_rag(query_text: str):
    embedding_function = get_embedding_function()
    db = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=embedding_function
    )
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    # print(prompt)

    model = OllamaLLM(model="llama3.2")
    response_text = model.invoke(prompt)

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text
    

In [5]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

query_text = "Can you explain the key features of SILOP?"
query_rag(query_text)

Response: According to the text, the key features of SILOP (Semantic Image Labeling using Object Perimeters) are:

1. An additional step between CAM (Class Activation Mapping) and pixel similarity-based refinement that improves prediction quality.
2. The incorporation of a PerimeterFit module, which can be used with any conventional CAM framework and existing WSSS pipelines.
3. Improved performance on certain classes, such as boat, person, and train, with gains of 4.9%, 3.7%, and 3.5% over PuzzleCAM, respectively.
4. Better performance in removing false positives and achieving more mIoU (mean Intersection over Union) points when successfully refining the segmentation mask.

Overall, SILOP appears to be an automated framework for semantic segmentation that leverages image-level labels based on object perimeters to improve prediction quality and robustness.
Sources: ['data\\SILOP.pdf:1:2', 'data\\SILOP.pdf:6:4', 'data\\SILOP.pdf:6:3', 'data\\SILOP.pdf:0:0', 'data\\SILOP.pdf:2:3']


'According to the text, the key features of SILOP (Semantic Image Labeling using Object Perimeters) are:\n\n1. An additional step between CAM (Class Activation Mapping) and pixel similarity-based refinement that improves prediction quality.\n2. The incorporation of a PerimeterFit module, which can be used with any conventional CAM framework and existing WSSS pipelines.\n3. Improved performance on certain classes, such as boat, person, and train, with gains of 4.9%, 3.7%, and 3.5% over PuzzleCAM, respectively.\n4. Better performance in removing false positives and achieving more mIoU (mean Intersection over Union) points when successfully refining the segmentation mask.\n\nOverall, SILOP appears to be an automated framework for semantic segmentation that leverages image-level labels based on object perimeters to improve prediction quality and robustness.'